<a href="https://colab.research.google.com/github/KONA1005/google_drive/blob/main/google_colab/Chapter_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치
- 키워드 : 하이퍼 파라미터
- 데이터가 작을 때 주로 사용
- 하이퍼파라미터 란?
 + max_depth : 3 으로 놨을때 정확도 84% 나왔어
 + 모르면 디폴트만 쓰자!!
 + 가성비 (시간 대비 성능 보장 안됨)

# 검증 세트
- 테스트 세트 (1회성임)
- 훈련데이터를 훈련데이터 + 검증데이터로 재 분할


## 현실
- 테스트 데이터가 별도로 존재 하지 않음
- 전체 데이터 = 훈련(6) + 검증(2) + 테스트(2)
  + 테스트 데이터는 모르는 데이터로 생각해야함

In [3]:
import pandas as pd
wine = pd.read_csv("https://bit.ly/wine_csv_data")

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2 , random_state=42
)

In [7]:
sub_input.shape , val_input.shape, test_input.shape

((4157, 3), (1040, 3), (1300, 3))

# 모델 만든 후 평가
- 과대적합 발생

In [9]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차 검증
- 목적 : 좋은 모델이 만들어 진다.
 + 좋은 모델 은 성능 좋은 모델이 아니다.
 + 좋은 모델은 과대 적합이 아닌 모델 **모형의 오차가 적은 모델 (안정적인 모델)**
- 245p
 + 모델 평가 1 : 정확도 90 / 소요시간 1시간
 + 모델 평가 2 : 정확도 85 
 + 모델 평가 3 : 정확도 80
- 유일한 단점 : 시간이 오래걸림.

# 교차 검증 함수

In [10]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)


{'fit_time': array([0.0111351 , 0.01115823, 0.01119614, 0.01256633, 0.01010013]), 'score_time': array([0.00580072, 0.00120902, 0.00114274, 0.00117636, 0.00116658]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- 최종 점수 평균 구하기

In [11]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


- 훈련 세트 섞은 후 10-폴드 교차검증

In [14]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits = 10, shuffle= True, random_state= 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)

print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼 파라미터 튜닝 꼭 하고 싶다?
- 랜덤 서치 사용하자
- 자동으로 잡아주는 라이브러리들이 등장하기 시작
 + 뭐 여러개 있음
 

In [16]:
from sklearn.model_selection import GridSearchCV
params = {
    'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
}
# 같음 dt = DecisionTreeClassifier(random_state = 42)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs= -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [19]:
dt = gs.best_estimator_
print(dt)
print(dt.score(train_input, train_target))
print(gs.best_params_)

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
0.9615162593804117
{'min_impurity_decrease': 0.0001}


In [20]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


# 랜덤 서치
- 252p. 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있도록 확률 분포 객체를 전달

In [22]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([7, 0, 8, 3, 1, 8, 1, 5, 0, 8])

In [23]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([111, 102, 106,  87, 105, 102,  93, 104, 101,  89]))

In [24]:
# 254p

from sklearn.model_selection import RandomizedSearchCV

params = {
    'min_impurity_decrease' : uniform(0.0001, 0.001),
    'max_depth' : randint(20,50)
}

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state =42)

gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f84fcf77ad0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f84fc52d490>},
                   random_state=42)

In [25]:
gs.best_params_

{'max_depth': 29, 'min_impurity_decrease': 0.000437615171403628}